In [93]:
import os
from glob import glob

from shapely.geometry import shape

import json

from hashlib import md5

from shutil import copyfile

from tqdm import tqdm

from subprocess import Popen

## Appendix: Compute Performance Metrics with Comparator Observations for Single ASO Collect

`run_prediction.py` produces a series of directories in S3, one per PlanetScope image, associated with a single trained model and ASO collection which the imagery overlaps (via the imagery specified in a model training `.toml` file).

`summarize.py` run over these data produce a series of directories, one per PlanetScope image, containing: 

* Prediction Tiles
* Merged Prediction Mask
* ASO Tiles
* Merged ASO Mask

The purpose of this notebook is to augment each of these directories with **clipped, contemporaneous, and co-located observations** from following three comparators: 

* ASO
* Landsat 8 fSCA
* Sentinel 2 NDSI

Each directory contains a contemporaneous, spatially overlapping ASO collect from `summarize.py`, so we only need to copy it. This notebook requires the location of each of the Landsat 8 and Sentinel 2 NDSI files. We assume that these overlapping data products have already been computed, and that a single comparator collect overlaps all imagery.

We use a file of selected PlanetScope images to constrain the processing. 


In [17]:
root = "/Volumes/wrangell-st-elias/research/planet/tuol-reruns-v2/ASO_3M_SD_USCATE_20180528/"

In [61]:
ls8_fsca = os.path.join(root, "LC08_CU_003009_20180603_20190615_C01_V01_SNOW/LC08_CU_003009_20180603_20190615_C01_V01_SNOW_05.tif")

In [62]:
s2_ndsi = os.path.join(root, "sentinel-2/11SKC,2018-05-27,0/NDSI.tif")

In [63]:
comparators = [ls8_fsca, s2_ndsi]

In [22]:
selected_image_ids = open(os.path.join(root, "selected_images_rework.txt")).read().split()

In [28]:
selected_images = [glob(os.path.join(root, f"*{i}*"))[0] for i in selected_image_ids]

In [26]:
selected_images

['/Volumes/wrangell-st-elias/research/planet/tuol-reruns-v2/ASO_3M_SD_USCATE_20180528/s3:::planet-snowcover-models:ASO-3M-SD-USCASJ-20180601-Step2-V2-2020-03-03-17-49-47-260:planet-snowcover-imagery:20180528_181108_1025_3B_AnalyticMS_SR_clip',
 '/Volumes/wrangell-st-elias/research/planet/tuol-reruns-v2/ASO_3M_SD_USCATE_20180528/s3:::planet-snowcover-models:ASO-3M-SD-USCASJ-20180601-Step2-V2-2020-03-03-17-49-47-260:planet-snowcover-imagery:20180528_181109_1025_3B_AnalyticMS_SR_clip',
 '/Volumes/wrangell-st-elias/research/planet/tuol-reruns-v2/ASO_3M_SD_USCATE_20180528/s3:::planet-snowcover-models:ASO-3M-SD-USCASJ-20180601-Step2-V2-2020-03-03-17-49-47-260:planet-snowcover-imagery:20180528_181110_1025_3B_AnalyticMS_SR_clip',
 '/Volumes/wrangell-st-elias/research/planet/tuol-reruns-v2/ASO_3M_SD_USCATE_20180528/s3:::planet-snowcover-models:ASO-3M-SD-USCASJ-20180601-Step2-V2-2020-03-03-17-49-47-260:planet-snowcover-imagery:20180528_181111_1025_3B_AnalyticMS_SR_clip',
 '/Volumes/wrangell-st-e

In [31]:
os.listdir(selected_images[0])

['data-mask.geojson', 'mask', 'preds']

First we create a directory unique to the files that are being combined and **add ASO to that directory**: 

In [98]:
selected_images_comparators = []
for image in selected_images:
    aso_collect_path = glob(os.path.join(image, "mask/*merged.tif"))[0]
    run_id = md5("_".join([ls8_fsca, s2_ndsi, aso_collect_path]).encode('utf-8')).hexdigest()
    selected_images_comparators.append(os.path.join(image, run_id))
    comparator_dir = os.path.join(image, run_id)
    os.makedirs(comparator_dir, exist_ok=True)
    copyfile(aso_collect_path, os.path.join(comparator_dir, "ASO_merged.tif"))

In [99]:
selected_images_comparators

['/Volumes/wrangell-st-elias/research/planet/tuol-reruns-v2/ASO_3M_SD_USCATE_20180528/s3:::planet-snowcover-models:ASO-3M-SD-USCASJ-20180601-Step2-V2-2020-03-03-17-49-47-260:planet-snowcover-imagery:20180528_181108_1025_3B_AnalyticMS_SR_clip/c8099c28f0abf143501bbcbfb84ba197',
 '/Volumes/wrangell-st-elias/research/planet/tuol-reruns-v2/ASO_3M_SD_USCATE_20180528/s3:::planet-snowcover-models:ASO-3M-SD-USCASJ-20180601-Step2-V2-2020-03-03-17-49-47-260:planet-snowcover-imagery:20180528_181109_1025_3B_AnalyticMS_SR_clip/00d2c3df253873185a3cbc45f6580342',
 '/Volumes/wrangell-st-elias/research/planet/tuol-reruns-v2/ASO_3M_SD_USCATE_20180528/s3:::planet-snowcover-models:ASO-3M-SD-USCASJ-20180601-Step2-V2-2020-03-03-17-49-47-260:planet-snowcover-imagery:20180528_181110_1025_3B_AnalyticMS_SR_clip/f9ed5fb7ea7e5d9a6f8a69b7d9513b9f',
 '/Volumes/wrangell-st-elias/research/planet/tuol-reruns-v2/ASO_3M_SD_USCATE_20180528/s3:::planet-snowcover-models:ASO-3M-SD-USCASJ-20180601-Step2-V2-2020-03-03-17-49-47

## Clip and Reproject, and Copy

Here we clip LS8 and S2 NDSI to a coarse bounding box of each PlanetScope image and re-project into a common EPSG, saving into the comparator directories we just generated. 

In [100]:
for i, image in enumerate(selected_images):
    mask = shape(json.loads(open(os.path.join(image, "data-mask.geojson")).read()))
    minx, miny, maxx, maxy = mask.bounds
    
    for comparator in comparators:
        projected = os.path.splitext(os.path.basename(comparator))[0] + "_reprojected.tif"
        projected_path = os.path.join(selected_images_comparators[i], projected)
        clipped =  os.path.splitext(os.path.basename(projected))[0] + "_clipped.tif"
        clipped_path = os.path.join(selected_images_comparators[i], clipped)
        
#         gdal_warp_cmd = f"source activate qgis && gdal_translate -a_nodata 9999 -projwin {minx} {maxy} {maxx} {miny} -projwin_srs EPSG:4326 {comparator} {clipped_path} && gdalwarp -dstnodata 9999 -overwrite -t_srs EPSG:4326 {clipped_path} {projected_path}" 
        gdal_warp_cmd = f"source activate qgis && gdalwarp -dstnodata 9999 -overwrite -t_srs EPSG:4326 {comparator} {projected_path} && gdal_translate -a_nodata 9999 -projwin {minx} {maxy} {maxx} {miny} -projwin_srs EPSG:4326 {projected_path} {clipped_path}" 
    
        Popen(gdal_warp_cmd, shell=True).communicate()
        os.remove(projected_path)


## Copy geojson into comparator

In [96]:
for i, image in enumerate(selected_images):
    # unfortunate mismatch in naming (data-mask.geojson to data_region.geojson). Other tools expect data_region. oops!
    copyfile(
        os.path.join(image, "data-mask.geojson"),
        os.path.join(image, selected_images_comparators[i], "data_region.geojson")
    )


## Copy ML predictions into comparator 

In [101]:
for i, image in enumerate(selected_images):
    pred_path = glob(os.path.join(image, "preds/*merged.tif"))[0]
    pred_fname = os.path.basename(pred_path)
    copyfile(
        pred_path, 
        os.path.join(image, selected_images_comparators[i], pred_fname)

    )


Should be good. 